[View in Colaboratory](https://colab.research.google.com/github/ShriPunta/Netflix-Graph-Dataset-Project/blob/master/ParsingTheCombinedFile.ipynb)

In [0]:
!pip install -U -q PyDrive


**Setup all the Authentication for PyDrive**

In [0]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build
import numpy as np



In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


---


** *Method*: To read the contents of a local Google File**



In [0]:
def read_drive_file_into_variable(file_id_to_read):
  drive_service = build('drive', 'v3')
  request = drive_service.files().get_media(fileId=file_id_to_read)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    status, done = downloader.next_chunk()
    prog = int(status.progress() * 100)
    print("Download "+str(prog))
  
  #Set the pointer to the start
  downloaded.seek(0)
  #print('Downloaded file contents are: {}'.format(downloaded.read()))
  
  #Read Everthing into a variable called "View", its in a "Bytes" datatype
  view = downloaded.read1(-1)
  #len(view)
  
  #decode Bytes to String format
  decoded = view.decode(encoding="utf-8")
  #type(decoded)
  
  #Convert this continous string to List for each new line
  variable_to_set = decoded.splitlines()
  #type(splitted)
  
  return variable_to_set
  
  

---

**Read the main two files into variables**

In [0]:
movie_Title_file_id = '0B8qgJkz0ynl8czBBNG9qZ2JPeW9RaWVuZktobUE1b29qaER3'
rating_file_id = '1iABtudmoCPxcFYiYQ0cQwfcQCZtfHsHd'
test_file_id = '11QhP0HwV7x6huJX3-JqJzGv7shMiZVPH'

In [6]:
key_file_list = read_drive_file_into_variable(movie_Title_file_id)


Download 100


In [7]:
rating_file_list = read_drive_file_into_variable(rating_file_id)

Download 21
Download 42
Download 63
Download 84
Download 100


In [0]:
#test_file_list = read_drive_file_into_variable(test_file_id)

---

**Convert the key file into a dataframe**

In [9]:
#Read the key file into a Panda dataframe
key_file_df = pd.DataFrame([sub.split(",") for sub in key_file_list],columns = ["MovieId","Year","Name","Genre1","Genre2","Genre3"])
key_file_df.head()

,MovieId,Year,Name,Genre1,Genre2,Genre3
0,1,2003,Dinosaur Planet,"""Animation",Documentary,"Family"""
1,3,1997,Character,"""Crime",Drama,"Mystery"""
2,6,1997,Sick,Drama,None,None
3,7,1992,8 Man,"""Action","Sci-Fi""",None
4,10,2001,Fighter,Documentary,None,None


**Clean the dataframe**

In [0]:
#key_file_df['Genre'] = key_file_df[['Genre1','Genre2','Genre3']].apply(lambda x: ''.join(x), axis=1)
# key_file_df.head()

def clean_the_movie_key_df(key_file_df):
  #Convert NaN to blank
  key_file_df = key_file_df.replace(np.nan, '', regex=True)
  
  
  #Combine the Genres into a single column
  key_file_df['Genre'] = key_file_df['Genre1'] + ',' + key_file_df['Genre2'] + ','+ key_file_df['Genre3']

  #Clean the column by removing double quotes
  key_file_df['Genre'] = key_file_df['Genre'].str.replace('"', '')

  #Drop the unnecessary columns
  key_file_df.drop(['Genre1','Genre2','Genre3'], axis=1, inplace=True)

  #Set MovieId as the Index
  key_file_df.set_index('MovieId',inplace=True)

  return key_file_df


In [0]:
key_file_df = clean_the_movie_key_df(key_file_df)

In [12]:
key_movie_values = key_file_df.index.values
key_movie_values[11]


'21'

---

In [0]:
def do_sampling(list_to_iterate):
  flag=True
  limiter = 100
  count = 0
  refined_list = []
  key_to_search = 0
  for ele in list_to_iterate:
    if ele.find(':') != -1:
      count = 1
      key_to_search = ele.replace(':','')
      if key_to_search in key_movie_values:
        flag = True
        refined_list.append(key_to_search)
      else:
        flag = False
    else:
      if count > limiter or flag is False:
        continue
      ele += ',' +str(key_to_search)
      refined_list.append(ele)
      count+=1
  return refined_list

In [0]:
refined_list = do_sampling(rating_file_list)


In [28]:
from sys import getsizeof
#print("Total number of elements -->",len(rating_file_list),"  Size they occupy in bytes -->",getsizeof(rating_file_list))

print("Total number of Elements in rating_file_list list -->",len(rating_file_list),"  Size they occupy in bytes -->",getsizeof(rating_file_list))
print("Total number of Elements in refined_list list -->",len(refined_list),"  Size they occupy in bytes -->",getsizeof(refined_list))

Total number of Elements in rating_file_list list --> 24058263   Size they occupy in bytes --> 209183992
Total number of Elements in refined_list list --> 280214   Size they occupy in bytes --> 2380488


---

**Convert rating File into dataframe**

In [31]:
#Read the key file into a Panda dataframe
test_file_df = pd.DataFrame([sub.split(",") for sub in refined_list],columns = ["UserId","Rating","YearWatched","MovieId"])

#Initialize a blank column called MovieId ; to be filled latter
#test_file_df["MovieId"] = 0
test_file_df[['MovieId']] = test_file_df[['MovieId']].apply(pd.to_numeric)

#Need memory optimization, convert the object type to numeric
test_file_df[['UserId']] = test_file_df[['UserId']].apply(pd.to_numeric)

#Need memory optimization, convert the object type to numeric
test_file_df[['Rating']] = test_file_df[['Rating']].apply(pd.to_numeric)

test_file_df['YearWatched'] = pd.to_datetime(test_file_df['YearWatched'])

#test_file_df[['YearWatched']] = test_file_df[['YearWatched']].apply(pd.to_datetime)
#test_file_df['YearWatched'] = pd.to_datetime(test_file_df['DateWatched']).dt.year


test_file_df['UserId'] = pd.to_numeric(test_file_df['UserId'],errors='ignore')

#Applying these changes in datatypes brought memory usage from 5.4 GB to 2 GB
test_file_df.info(memory_usage = 'deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280214 entries, 0 to 280213
Data columns (total 4 columns):
UserId         280214 non-null int64
Rating         277420 non-null float64
YearWatched    277420 non-null datetime64[ns]
MovieId        277420 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 8.6 MB


In [39]:
test_file_df.head(5)

,UserId,Rating,YearWatched,MovieId
0,1,NaN,NaT,NaN
1,1488844,3.0,2005-09-06,1.0
2,822109,5.0,2005-05-13,1.0
3,885013,4.0,2005-10-19,1.0
4,30878,4.0,2005-12-26,1.0


In [45]:
the_NaN_matrix = test_file_df.isnull()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280214 entries, 0 to 280213
Data columns (total 4 columns):
UserId         280214 non-null bool
Rating         280214 non-null bool
YearWatched    280214 non-null bool
MovieId        280214 non-null bool
dtypes: bool(4)
memory usage: 1.1 MB


In [46]:
the_NaN_matrix.head()

,UserId,Rating,YearWatched,MovieId
0,False,True,True,True
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False


---

In [0]:
#We will store the rows to drop in this
rows_to_drop = []
currentMovieId = 0
list_movieId = []


#Iterate over the dataframe to split it
for index, row in test_file_df.iterrows():
  if the_NaN_matrix.iloc[index,2]:
    #If entered it means that there is a colon on the row and it is a movie id
    #row gives the first character on that row
    #currentMovieId = row['UserId']
    rows_to_drop.append(index)
  else:
    list_movieId.append(currentMovieId)
    #print("Not")
    #Assign the movie id to the consecutive rows
    #test_file_df.iloc[index,3] = currentMovieId

#Create a series object from the list
MovieIdSeries = pd.Series(list_movieId)  

In [18]:
#We get a dataframe with the movie id generated, now we need to filter out the rows
test_file_df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280214 entries, 0 to 280213
Data columns (total 4 columns):
UserId         280214 non-null int64
Rating         277420 non-null float64
YearWatched    277420 non-null datetime64[ns]
MovieId        280214 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 8.6 MB


In [0]:
#Drop the rows which had the movie Id and the columns to get a seamless dataframe
test_file_df.drop(test_file_df.index[rows_to_drop],inplace=True)


In [20]:
test_file_df.head()

,UserId,Rating,YearWatched,MovieId
0,1,NaN,NaT,0
1,1488844,3.0,2005-09-06,0
2,822109,5.0,2005-05-13,0
3,885013,4.0,2005-10-19,0
4,30878,4.0,2005-12-26,0


In [0]:
test_file_df['MovieId'] =  MovieIdSeries

In [0]:
#Convert the column to datetime and keep only the year
test_file_df['YearWatched'] = pd.to_datetime(test_file_df['YearWatched']).dt.year

In [23]:
test_file_df.head()

,UserId,Rating,YearWatched,MovieId
0,1,NaN,NaN,0
1,1488844,3.0,2005.0,0
2,822109,5.0,2005.0,0
3,885013,4.0,2005.0,0
4,30878,4.0,2005.0,0


---
 ** *Method*: To see the GPU usage**

In [0]:
def checkGPU():
  # memory footprint support libraries/code
  !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
  !pip install gputil
  !pip install psutil
  !pip install humanize
  import psutil
  import humanize
  import os
  import GPUtil as GPU
  GPUs = GPU.getGPUs()
  # XXX: only one GPU on Colab and isn’t guaranteed
  gpu = GPUs[0]
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  


---

In [25]:
checkGPU()

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /content/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
  Running setup.py bdist_wheel for humanize ... - done
  Stored in directory: /content/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 8.5 GB  I Proc size: 4.4 GB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB
